<a href="https://colab.research.google.com/github/MehrdadJalali-AI/Data_Management/blob/main/handle_non_relational_xml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Handling Non-relational Data in XML Format

## Summary


        This notebook demonstrates solutions for handling non-relational XML data, including:
        - Parsing XML into a DataFrame.
        - Validating schema and adding missing columns.
        - Handling missing values by filling them with defaults or flagging them.
        - Enforcing data types (e.g., numeric, string).
        - Validating patterns (e.g., valid email formats).
        - Checking completeness for required fields.
        

In [1]:

# Step 1: Parse XML Data
import pandas as pd
from lxml import etree

# Sample XML Data
xml_data = '''
<root>
    <record>
        <CustomerID>1001</CustomerID>
        <Name>John Doe</Name>
        <Age>35</Age>
        <Email>john.doe@email.com</Email>
    </record>
    <record>
        <CustomerID>1002</CustomerID>
        <Name>Jane Smith</Name>
        <Email>jane.smith@email.com</Email>
    </record>
    <record>
        <CustomerID>1003</CustomerID>
        <Name>Alice Johnson</Name>
        <Age>Thirty</Age>
        <Email>alice.johnson#email</Email>
    </record>
    <record>
        <CustomerID>1004</CustomerID>
        <Name>Bob Brown</Name>
        <Age>42</Age>
    </record>
</root>
'''

def parse_xml(xml_string):
    root = etree.fromstring(xml_string)
    records = []
    for record in root.findall("record"):
        record_data = {child.tag: child.text for child in record}
        records.append(record_data)
    return pd.DataFrame(records)

df = parse_xml(xml_data)
df


,CustomerID,Name,Age,Email
0,1001,John Doe,35,john.doe@email.com
1,1002,Jane Smith,NaN,jane.smith@email.com
2,1003,Alice Johnson,Thirty,alice.johnson#email
3,1004,Bob Brown,42,NaN


In [2]:

# Step 2: Schema Validation
required_columns = ["CustomerID", "Name", "Age", "Email"]

def validate_schema(df, required_columns):
    for col in required_columns:
        if col not in df.columns:
            df[col] = None  # Add missing columns with None values
    return df

df = validate_schema(df, required_columns)
df


,CustomerID,Name,Age,Email
0,1001,John Doe,35,john.doe@email.com
1,1002,Jane Smith,NaN,jane.smith@email.com
2,1003,Alice Johnson,Thirty,alice.johnson#email
3,1004,Bob Brown,42,NaN


In [3]:

# Step 3: Handle Missing Values
default_values = {"Age": 0, "Email": "unknown@email.com"}

def handle_missing_values(df, default_values):
    for col, default in default_values.items():
        df[col] = df[col].fillna(default)
    return df

df = handle_missing_values(df, default_values)
df


,CustomerID,Name,Age,Email
0,1001,John Doe,35,john.doe@email.com
1,1002,Jane Smith,0,jane.smith@email.com
2,1003,Alice Johnson,Thirty,alice.johnson#email
3,1004,Bob Brown,42,unknown@email.com


In [4]:

# Step 4: Enforce Data Types
type_map = {"CustomerID": "int", "Age": "int", "Email": "str"}

def enforce_data_types(df, type_map):
    for col, dtype in type_map.items():
        if dtype == "int":
            df[col] = pd.to_numeric(df[col], errors='coerce')  # Convert to numeric, set invalid to NaN
        elif dtype == "str":
            df[col] = df[col].astype(str)
    return df

df = enforce_data_types(df, type_map)
df


,CustomerID,Name,Age,Email
0,1001,John Doe,35.0,john.doe@email.com
1,1002,Jane Smith,0.0,jane.smith@email.com
2,1003,Alice Johnson,NaN,alice.johnson#email
3,1004,Bob Brown,42.0,unknown@email.com


In [5]:

# Step 5: Validate Patterns
pattern_map = {"Email": r"^\S+@\S+\.\S+$"}

def validate_patterns(df, pattern_map):
    errors = {}
    for col, pattern in pattern_map.items():
        invalid = ~df[col].str.match(pattern, na=False)
        errors[col] = df[invalid]
        df.loc[invalid, col] = None  # Replace invalid values with None
    return df, errors

df, pattern_errors = validate_patterns(df, pattern_map)
df


,CustomerID,Name,Age,Email
0,1001,John Doe,35.0,john.doe@email.com
1,1002,Jane Smith,0.0,jane.smith@email.com
2,1003,Alice Johnson,NaN,None
3,1004,Bob Brown,42.0,unknown@email.com


In [6]:

# Step 6: Check Completeness
required_columns = ["CustomerID", "Name", "Email"]

def check_completeness(df, required_columns):
    missing_rows = df[df[required_columns].isnull().any(axis=1)]
    return missing_rows

missing_data = check_completeness(df, required_columns)
missing_data


,CustomerID,Name,Age,Email
2,1003,Alice Johnson,NaN,None
